### RAG结合LLM

不用RAG给LLM灌输上下文数据

In [1]:
from langchain_openai import ChatOpenAI
import os
import dotenv
dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model="gpt-4o-mini")

# 调用
response = llm.invoke("北京有什么著名的建筑？")
print(response.content)

/opt/anaconda3/envs/langchain/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


北京作为中国的首都，拥有许多著名的建筑和历史遗迹。以下是一些代表性的建筑：

1. **故宫**：明清两代的皇家宫殿，是中国现存最完整、规模最大的一组古代建筑群，也是世界文化遗产。

2. **天安门广场**：中国最大的城市广场，广场上有毛主席纪念堂、人民大会堂等重要建筑。

3. **天坛**：明清时期的皇家祭天场所，以其独特的建筑风格和大的圆形设计而闻名，也是世界文化遗产。

4. **颐和园**：清代皇家园林，以其湖光山色和丰富的文化遗产著称，是游览和休闲的理想去处。

5. **长城**：虽然长城跨越多个省份，但在北京附近的八达岭和慕田峪段非常著名，是中国古代防御工程的象征。

6. **鸟巢（国家体育场）**：为2008年北京奥运会而建，是现代建筑的代表之一。

7. **水立方（国家游泳中心）**：同样为2008年奥运会而建，以其独特的“水泡”设计而受到广泛关注。

8. **798艺术区**：原为工业区，现已转型为艺术区，聚集了许多画廊、艺术工作室和创意产业。

9. **北京大学和清华大学**：这两所大学不仅学术声誉高，校园内的建筑和环境也极具特色。

这些建筑和遗迹展示了北京丰富的历史和文化，是游客和居民必访的地方。


**情况2：使用RAG给LLM灌输上下文数据**

In [2]:
# 1. 导入所有需要的包
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
import os
import dotenv

dotenv.load_dotenv()

# 2. 创建自定义提示词模板
prompt_template = """请使用以下提供的文本内容来回答问题。仅使用提供的文本信息，如果文本中没有相关信息，请回答"抱歉，提供的文本中没有这个信息"。

文本内容：
{context}

问题：{question}

回答：
"
"""

prompt = PromptTemplate.from_template(prompt_template)

# 3. 初始化模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 4. 加载文档
loader = TextLoader("./asset/load/10-test_doc.txt", encoding='utf-8')
documents = loader.load()

# 5. 分割文档
text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    length_function=len
)
texts = text_splitter.split_documents(documents)

print(f"文档个数:{len(texts)}")

# 6. 创建向量存储
vectorstore = FAISS.from_documents(
    documents=texts,
    embedding=embeddings
)

# 7.获取检索器
retriever = vectorstore.as_retriever()

docs = retriever.invoke("北京有什么著名的建筑？")

# 8. 创建Runnable链
chain = prompt | llm

# 9. 提问
result = chain.invoke(input={"question":"北京有什么著名的建筑？","context":docs})
print("\n回答:", result.content)

文档个数:1

回答: 北京有以下著名的建筑：

1. 故宫 - 明清两代的皇家宫殿，世界上现存规模最大、保存最完整的木质结构古建筑群之一。
2. 天安门 - 北京的标志性建筑之一，天安门广场是世界上最大的城市广场。
3. 颐和园 - 清朝时期的皇家园林，融合了江南园林的设计风格。
4. 天坛 - 明清两代皇帝祭天、祈谷的场所，具有深厚的文化内涵。
5. 长城（八达岭段） - 最著名的北京段，被誉为"世界第八大奇迹"。
6. 国家体育场（鸟巢） - 2008年奥运会主体育场，以独特的钢结构设计著称。
7. 中央电视台总部大楼 - 现代北京最具争议和识别度的建筑之一。
8. 国家大剧院 - 因其蛋壳造型被称为"巨蛋"，是世界最大的穹顶建筑之一。
9. 北京大兴国际机场 - 超大型国际航空枢纽，被誉为"新世界七大奇迹"之一。
10. 鼓楼和钟楼 - 古代中国的计时中心，展现了古代的计时智慧。


### 项目：智能对话助手

#### 1  定义工具

In [3]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults

# 查询 Tavily 搜索 API
search = TavilySearchResults(max_results=1)
# 执行查询
res = search.invoke("今天上海天气怎么样")
print(res)

/var/folders/88/h2503n817r15dm_yghgn_vcm0000gn/T/ipykernel_92984/2622377732.py:5: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults(max_results=1)


[{'title': '上海, 上海市, 中國每月天氣', 'url': 'https://www.accuweather.com/zh/cn/shanghai/106577/august-weather/106577', 'content': '上海, 上海市, 中國 每月天氣 | AccuWeather 上海, 上海市 ======= 72°FImage 1 上海 上海市 72° 上海, 上海市 天氣 27 90° 79°28 91° 78°29 91° 77°30 90° 77°31 90° 78°1 91° 78°2 90° 75°3 89° 77°4 88° 76°5 87° 75°6 88° 76°7 87° 79°8 87° 79°9 88° 78°10 87° 79°11 86° 79°12 86° 78°13 88° 77°14 89° 78°15 88° 77°16 86° 76°17 86° 77°18 85° 77°19 85° 76°20 86° 78°21 85° 78°22 86° 77°23 86° 77°24 85° 76°25 84° 75°26 85° 76°27 84° 73°28 84° 75°29 83° 72°30 83° 72°31 84° 71°1 83° 72°2 84° 73°3 83° 75°4 82° 73°5 83° 75°6 82° 71° *   嘉興, 浙江省 *   寧波, 浙江省 © 2025 AccuWeather, Inc. 版權所有。「AccuWeather」和太陽商標設計是 AccuWeather, Inc. 的註冊商標。保留所有權利。  Get AccuWeather alerts as they happen with our browser notifications. Enable Notifications Notifications Enabled', 'score': 0.9854}]


#### 2 Retriever

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import dotenv
dotenv.load_dotenv()

# 1. 提供一个大模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

embedding_model = OpenAIEmbeddings()

# 2.加载HTML内容为一个文档对象
loader = WebBaseLoader("https://zh.wikipedia.org/wiki/%E7%8C%AB")
docs = loader.load()
#print(docs)

# 3.分割文档
documents = RecursiveCharacterTextSplitter(
    #chunk_size 参数在 RecursiveCharacterTextSplitter 中用于指定每个文档块的最大字符数。它的作用主要有以下几个方面：
    #chunk_overlap 参数用于指定每个文档块之间的重叠字符数。这意味着，当文档被拆分成较小的块时，每个块的末尾部分会与下一个块的开头部分有一定数量的重叠字符。
    #第一个块包含字符 1 到 1000。第二个块包含字符 801 到 1800。第三个块包含字符 1601 到 2600。
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

# 4.向量化 得到向量数据库对象
vector = FAISS.from_documents(documents, embedding_model)

# 5.创建检索器
retriever = vector.as_retriever()

# 测试检索结果
print(retriever.invoke("猫的特征")[0])

USER_AGENT environment variable not set, consider setting it to identify your requests.


page_content='感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[48]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[49][50]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。' metadata={'source': 'https://zh.wikipedia.org/wiki/%E7%8C%AB', 'title': '猫 - 维基百科，自由的百科全书'

#### 3 创建工具、工具集

In [5]:
from langchain.tools.retriever import create_retriever_tool
# 创建一个工具来检索文档
retriever_tool = create_retriever_tool(
    retriever,
    "wiki_search",
    "搜索维基百科",
)
tools = [search, retriever_tool]

#### 4 语言模型调用工具

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
model = ChatOpenAI(model="gpt-4")

# 模型绑定工具
model_with_tools = model.bind_tools(tools)

# 根据输入自动调用工具
response = model_with_tools.invoke([HumanMessage(content="今天上海天气怎么样")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '今天上海天气'}, 'id': 'call_Yng06yYSUQnfjZrozIqZUtXh', 'type': 'tool_call'}]


#### 5 创建Agent程序(使用通用方式)

In [7]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [11]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor
# 创建Agent对象
agent = create_tool_calling_agent(model, tools, prompt)

# 创建AgentExecutor对象
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)

In [12]:
print(agent_executor.invoke({"input": "猫的特征"}))



> Entering new AgentExecutor chain...

Invoking: `wiki_search` with `{'query': '猫的特征'}`


感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[48]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[49][50]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。

貓爪[编辑]
貓的爪子尖

In [13]:
print(agent_executor.invoke({"input": "今天上海天气怎么样"}))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '今天上海天气'}`


[{'title': '上海 - 百度', 'url': 'https://weathernew.pae.baidu.com/weathernew/pc?query=%E4%B8%8A%E6%B5%B7%E5%A4%A9%E6%B0%94&srcid=4982&forecast=long_day_forecast', 'content': '===============\n\nImage 1\n\n上海 今天：多云  21°~31°C  南风1级\n\n06月04日 周三 农历五月初九\n\n27°\n\n104 轻度\n\n多云 东风  1级 \n\n湿度57%日出04:50 日落18:55\n\n24小时预报\n\n    温度 \n    降水量 \n    风力风向 \n    空气质量 \n\n31°C 26°C 21°C\n\n现在\n\n21:00\n\n22:00\n\n23:00\n\n00:00\n\n01:00\n\n02:00\n\n03:00\n\n04:00\n\n05:00\n\n06:00\n\n07:00\n\n08:00\n\n09:00\n\n10:00\n\n11:00\n\n12:00\n\n13:00\n\n14:00\n\n15:00\n\n16:00\n\n17:00\n\n18:00\n\n19:00\n\n15天天气预报\n\n   昨天\n\n06/03\n\n1级\n\n优\n\n   今天\n\n06/04\n\n1级\n\n轻度\n\n   明天\n\n06/05\n\n3级\n\n轻度\n\n   周五\n\n06/06\n\n3级\n\n轻度\n\n   周六\n\n06/07\n\n3级\n\n良\n\n   周日\n\n06/08\n\n1级\n\n良\n\n   周一\n\n06/09\n\n1级', 'score': 0.7738498}]今天上海的天气是多云，温度在21°~31°C之间，风力为南风1级。空气质量为轻度污染。

> Finished chain.
{'input'

#### 添加记忆

In [14]:
from langchain_community.chat_message_histories import ChatMessageHistory

from langchain_core.chat_history import BaseChatMessageHistory

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# 调取指定session_id对应的memory
def get_session_history(session_id: str) -> BaseChatMessageHistory:

    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)
response = agent_with_chat_history.invoke(
    {"input": "Hi，我的名字是Cyber"},
    config={"configurable": {"session_id": "123"}},
)

print(response)



> Entering new AgentExecutor chain...
你好，Cyber！很高兴认识你。有什么我可以帮助你的吗？

> Finished chain.
{'input': 'Hi，我的名字是Cyber', 'chat_history': [], 'output': '你好，Cyber！很高兴认识你。有什么我可以帮助你的吗？'}
